# Qiskit Pulseを用いたJaynes-Cummingsモデルの探求

### 物理的な背景

Jaynes-Cummingsモデルは2準位系(量子ビット)と単一モードのキャビティ(共振器)との相互作用を説明します。2準位系をキャビティに配置すると強度$g$で結合し、$\gamma$レートで励起状態から自然放出します。一方キャビティは$\kappa$で緩和します。このチュートリアルでは、超伝導量子ビットと超伝導共振器で構成される系のパラメーターをQiskit Pulseを使って測定します。

<img src="images/CQED.png" width="250"/>

この量子ビットーキャビティ間の相互作用はJaynes-Cummingsハミルトニアンで説明できます。

$H_{JC}/\hbar=\omega_r(a^\dagger a) - \frac{1}{2} \omega_q \sigma_z + g (\sigma_+ a + \sigma_- a^\dagger)$

まずは、このハミルトニアンを分解して観察しましょう。ハミルトニアンの1項目、$H_r/\hbar=\omega_r(a^\dagger a)$は共振器を表現しています。共振器は量子調和振動子とみなすことができ、 $\omega_r$ は共振周波数、$a$と$a^\dagger$ は共振気中の光子の昇降演算子です。簡単のために調和振動子のゼロ点エネルギーを無視していることに注意してください。ハミルトニアンの2項目$H_q/\hbar=-\frac{1}{2} \omega_q \sigma_z$は量子ビットを表しています。
ここで$\omega_q$は量子ビットの周波数、$\sigma_z$ はパウリZ演算子です。最後の項$H_{rq}/\hbar=g (\sigma_+ a + \sigma_- a^\dagger)$は共振器と量子ビットの相互作用を表しています。 $g$ は量子ビットと共振器の結合強度、演算子$\sigma_+$と$\sigma_-$は量子ビットの生成消滅演算子です。この相互作用項に基づいて、我々は量子ビットの励起が共振器中の光子の損失を導く過程を観察することができます。逆もまた同じです。

量子ビットと共振器の周波数の離調$\Delta=\omega_q-\omega_r$は両者の結合強度よりも小さいです($|\Delta|\ll g$)。それゆえ、共振器ー量子ビット系は混合状態となり、2量子ビット操作に有用なコヒーレントな励起とスワップを導きます。しかしながら、光学的読み出しのために、我々は分散条件下で系の制御するため、量子ビットー共振器の周波数の離調は結合率や共振器の緩和率$|\Delta| \gg g,\kappa$よりもかなり大きいです。この条件下では量子ビットと共振器間の相互作用はお互いの周波数に影響を及ぼし、その特性は量子ビットの測定に利用されます。共振器中の光子が数個という条件下では、分散近似を適用することができ、ハミルトニアンは2次の摂動として扱うことができます。

$H_{JC(disp)}/\hbar=(\omega_r+ \chi \sigma_z) a^\dagger a + \frac{1}{2} \tilde{\omega}_q \sigma_z$

ここで$\chi=-g^2/\Delta$は分散シフト(負の符号はトランズモンが負の非調和性を保持しているためです)で、$\tilde{\omega}_q= \omega_q+g^2/\Delta$ はラムシフトが適用された修正された量子ビットの周波数です。

量子回路電磁気学の導出については他にも<a href="https://qiskit.org/textbook/ch-quantum-hardware/cQED-JC-SW.html">こちらの章</a>で議論されています。 

### 0. はじめに

最初に基本的な依存関係とヘルパー関数を準備します。

In [ ]:
# Qiskitの標準ライブラリのと構成をインポート
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
#IBM Quantum Experienceのアカウントを読み込む
provider = IBMQ.load_account()
IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

### 注意
この実験は現在の`ibmq_armonk`デバイスでは測定装置の構成上利用できません。妥当なスキャンレンジでは共振を見つけることはできないため、アクセス可能であれば他のデバイスを利用してください。


In [ ]:
backend.configuration().backend_version

続いて、デフォルトのバックエンド構成と設定を準備します。

In [ ]:
backend_config = backend.configuration()
backend_defaults = backend.defaults()

次にフィッティングとデータ分析のための複数のヘルパー関数を定義します。

In [ ]:
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

# サンプルはハードウェア制約により16の倍数である必要がある
def get_closest_multiple_of_16(num):
    return int(num + 8 ) - (int(num + 8 ) % 16)

# 反射測定の結果を処理する
# 反射測定のデータは、シグナル出力信号の位相でエンコードされている
def process_reflective_measurement(freqs, values):
    phase_grad = np.gradient(savgol_filter(np.unwrap(np.angle(values)), 3, 2), freqs)
    return (phase_grad-min(phase_grad))/(max(phase_grad)-min(phase_grad)) - 1

# ローレンツ関数
def lorentzian(f, f0, k, a, offs):
    return -a*k/(2*np.pi)/((k/2)**2+(f-f0)**2)+offs

# fit_lorentzianは周波数とそれぞれの周波数に対応する実験結果の２つの配列を引数として受け取る
# 実験結果に最もよくフィットするローレンツパラメータを返す
# poptはフィッティングパラメータでpconvはフィットの共分散行列
def fit_lorentzian(freqs, values):
    p0=[freqs[np.argmin(values)], (freqs[-1]-freqs[0])/2, min(values), 0]
    bounds=([freqs[0], 0, -np.inf, -np.inf], [freqs[-1], freqs[-1]-freqs[0], np.inf, np.inf])
    popt, pcov=curve_fit(lorentzian, freqs, values, p0=p0, bounds=bounds)
    return popt, pcov

# 実験関数
def exponential(t, tau, a, offset):
    return a*np.exp(-t/tau)+offset

# 指数関数でフィットする
def fit_exponential(ts, values):
    p0=[np.average(ts), 1, 0]
    return curve_fit(exponential, ts, values, p0=p0)

### 1. $\kappa$の測定

不完全な電磁気キャビティでは光子は緩和します。共振器の緩和率$\kappa$は共振器のスペクトロスコピーによる共振器ピークの線幅を計算することによって測定できます。大きな$\kappa$はより大きな損失のある共振器であることを意味しています。共振器の損失は品質係数$Q=\omega_r/\kappa$として保証されます。高い$Q$値はキャビティのエネルギー損失が少ないことを意味します。

In [ ]:
from qiskit import pulse            # パルスライブラリ
from qiskit.pulse import Play, Acquire
from qiskit.circuit import Parameter      #　あらゆるパラメータのパラメータクラス
import numpy as np

qubit=0   # 実験で利用する量子ビット

readout_time = 4e-6
readout_sigma = 10e-9 

# 読み出し出力信号の取得設定
acquisition_time = readout_time   # 全期間の読み出しの読み込み信号を取得したい

In [ ]:
center_freq = backend_defaults.meas_freq_est[qubit]  # 共振周波数の評価
freq_span = 0.3e6 # 共振器のスキャン区間。スキャン区間は共振器の線幅kappaよりも大きくする

frequencies_range = np.linspace(center_freq-freq_span/2, center_freq+freq_span/2, 41)

In [ ]:
# 低出力の共振器スペクトロスコピーのための連続パルスの準備
freq = Parameter('freq')
amp = Parameter('amp')
with pulse.build(backend=backend, name='readout_cavity_spectroscopy') as meas_spect_sched:
    acq_chan = pulse.acquire_channel(qubit)
    meas_chan = pulse.measure_channel(qubit)
    pulse.set_frequency(freq, meas_chan)
    # ガウス型の立ち上がりトと立ち下がり時間を持つスクエアパルスを使用
    duration = get_closest_multiple_of_16(pulse.seconds_to_samples(readout_time))
    sigma = get_closest_multiple_of_16(pulse.seconds_to_samples(readout_sigma))
    width = get_closest_multiple_of_16(pulse.seconds_to_samples(readout_time-8*readout_sigma))
    # 共振器のドライブ
    pulse.play(pulse.GaussianSquare(duration = duration,
                                    amp = amp,
                                    sigma = sigma,
                                    width = width,
                                    name = 'readout tone'), meas_chan)
    # 読み出し信号を取得する
    pulse.acquire(duration = get_closest_multiple_of_16(pulse.seconds_to_samples(acquisition_time)),
                  qubit_or_channel = acq_chan,
                  register = pulse.MemorySlot(qubit))

In [ ]:
low_power_schedules = [meas_spect_sched.assign_parameters({freq: f, amp: .3}, inplace=False) for f in frequencies_range]

ここでハードウェアに連続パルスを照射する。

In [ ]:
from qiskit.tools.monitor import job_monitor

num_shots_per_frequency = 2*1024

job_low_power = backend.run(low_power_schedules, 
                            meas_level=1, 
                            meas_return='avg', 
                            shots=num_shots_per_frequency)
job_monitor(job_low_power)

low_power_sweep_results = job_low_power.result(timeout=120)

In [ ]:
import matplotlib.pyplot as plt

low_power_sweep_values = []
for i in range(len(low_power_sweep_results.results)):
    res_low_power = low_power_sweep_results.get_memory(i)
    low_power_sweep_values.append(res_low_power[qubit])

low_power_sweep_values = process_reflective_measurement(frequencies_range, low_power_sweep_values)

plt.plot(frequencies_range/1e3, low_power_sweep_values, '-o', color='red', lw=2)

popt_low_power, _=fit_lorentzian(frequencies_range, low_power_sweep_values)

popt_low_power, _=fit_lorentzian(frequencies_range, low_power_sweep_values)
f0, kappa, a, offset = popt_low_power

fs=np.linspace(frequencies_range[0], frequencies_range[-1], 1000)
plt.plot(fs/1e3, lorentzian(fs, *popt_low_power), color='red', ls='--')
plt.annotate("", xy=((f0-kappa/2)/1e3, offset-1/2), xytext=((f0+kappa/2)/1e3, offset-1/2), arrowprops=dict(arrowstyle="<->", color='black'))
plt.annotate("$\kappa$={:d} kHz".format(int(kappa/1e3)), xy=((f0-kappa/2)/1e3, offset-.45), color='black')

plt.grid()
plt.xlabel("Frequency [kHz]")
plt.ylabel("Measured signal [a.u.]")
plt.show()

### 2. $\chi$と$g$の測定

次に、量子ビットー共振器間の結合を測定します。分散シフト ($\chi$)とそれに伴う量子ビットー共振器間の結合($g=\sqrt{\chi.\Delta}$)を測定する一つの手法は、分散条件下の共振周波数と$\chi$シフトが発生しない非相互作用領域の周波数を比較することです。非相互作用領域下では共振器の光子数$n=a^\dagger a$は$n_c=\frac{\Delta^2}{4g^2}$よりも大きいです。実験では高出力で共振器をドライブすることによって、多くの光子で共振器を充たすことができます。

In [ ]:
schedule_frequencies = [meas_spect_sched.assign_parameters({freq: f, amp: 1}, inplace=False) for f in frequencies_range]

ここでハードウェアに連続パルスを照射する。

In [ ]:
from qiskit.tools.monitor import job_monitor

num_shots_per_frequency = 2*1024

job_low_power = backend.run(low_power_schedules, 
                            meas_level=1, 
                            meas_return='avg', 
                            shots=num_shots_per_frequency)
job_monitor(job_low_power)

low_power_sweep_results = job_low_power.result(timeout=120)

そして次に高出力の共振器スペクトロスコピーに関する測定データにアクセスします

In [ ]:
high_power_sweep_values = []
for i in range(len(high_power_sweep_results.results)):
    res_high_power = high_power_sweep_results.get_memory(i)
    high_power_sweep_values.append(res_high_power[qubit])

high_power_sweep_values = process_reflective_measurement(frequencies_range, high_power_sweep_values)

popt_high_power, _=fit_lorentzian(frequencies_range, high_power_sweep_values)

最後に高出力の共振器スペクトロスコピーを低出力共振器スペクトロスコピーの横にプロットし、共振周波数のシフトを使って$\chi$を計算します。

In [ ]:
plt.plot(frequencies_range/1e3, high_power_sweep_values, '-o', color='black', lw=2, label='non-interactive')
plt.plot(frequencies_range/1e3, low_power_sweep_values, '-o', color='red', lw=2, label='dispersive')

fs=np.linspace(frequencies_range[0], frequencies_range[-1],1000)
plt.plot(fs/1e3, lorentzian(fs, *popt_high_power), color='black', ls='--')
plt.plot(fs/1e3, lorentzian(fs, *popt_low_power), color='red', ls='--')

plt.axvline(x=popt_low_power[0]/1e3, color='red')
plt.axvline(x=popt_high_power[0]/1e3, color='black')

chi=popt_low_power[0]-popt_high_power[0]
plt.annotate("", xy=(popt_low_power[0]/1e3, -.1), xytext=(popt_high_power[0]/1e3, -.1), arrowprops=dict(arrowstyle="<->", color='black'))
plt.annotate("$\chi$={:d} kHz".format(int(chi/1e3)), xy=(popt_high_power[0]/1e3, -.05), color='black')

plt.grid()
plt.xlabel("Frequency [kHz]")
plt.ylabel("Measured signal [a.u.]")
plt.legend()
plt.show()

print(r'chi={:.1f} kHz'.format((popt_low_power[0]-popt_high_power[0])/1e3))
Delta=abs(backend_defaults.meas_freq_est[qubit] - backend_defaults.qubit_freq_est[qubit])
print(r'g={:.1f} MHz'.format(np.sqrt(chi*Delta)/1e6))

### 3. $\gamma$の測定

共振器に結合した量子ビットはキャビティ内に光子を自発的に放出し、励起状態から基底状態へと緩和します。光子の自発的放出は量子ビットの環境により促進されますが、この現象をパーセル効果と呼びます。マイクロ波を量子ビットにドライブすることで量子ビットを励起させて、緩和率$T_1=1/\gamma$を測定することで、量子ビットの緩和率$\gamma$を測定できます。この実験は<a href="https://qiskit.org/textbook/ch-quantum-hardware/cQED-JC-SW.html">こちら章</a>で議論されているように、量子ビットのコヒーレンス特性を測定する一般的な方法です。この実験ではマイクロ波ドライブは$\pi$パルスである必要はありません。

In [ ]:
drive_sigma = 100e-9
drive_duration = 8*drive_sigma

# 量子ビットをドライブして一定時間待機後、量子ビットを測定する
# この測定をすることで量子ビットの占有率と遅延時間のプロットを作成できる
delay_times=np.linspace(0, 600e-6, 61) #measurement time delays
qubit_decay_pulses = []
for delay in delay_times:
    with pulse.build(backend=backend, default_alignment='sequential', name=f"decay delay = {delay * 1e6} us") as temp_decay_pulse:
        drive_chan = pulse.drive_channel(qubit)
        meas_chan = pulse.measure_channel(qubit)
        acq_chan = pulse.acquire_channel(qubit)

        # 量子ビットをドライブする
        pulse.play(pulse.Gaussian(duration=get_closest_multiple_of_16(pulse.seconds_to_samples(drive_duration)),
                                  amp=.5,
                                  sigma=get_closest_multiple_of_16(pulse.seconds_to_samples(drive_sigma)),
                                  name='qubit tone'), drive_chan)
        # 量子ビット測定前に遅延時間分待機する
        pulse.delay(get_closest_multiple_of_16(pulse.seconds_to_samples(delay)), meas_chan)
                
        with pulse.align_left():
            duration = get_closest_multiple_of_16(pulse.seconds_to_samples(readout_time))
            sigma = get_closest_multiple_of_16(pulse.seconds_to_samples(readout_sigma))
            width = get_closest_multiple_of_16(pulse.seconds_to_samples(readout_time-8*readout_sigma))
            # 低出力で共振器をドライブする
            pulse.play(pulse.GaussianSquare(duration = duration,
                                            amp = .3,
                                            sigma = sigma,
                                            width = width,
                                            name = 'low power readout tone'), meas_chan)
            # 読み出し信号を取得する
            pulse.acquire(duration = get_closest_multiple_of_16(pulse.seconds_to_samples(acquisition_time)),
                          qubit_or_channel = acq_chan,
                          register = pulse.MemorySlot(qubit))
        
    qubit_decay_pulses.append(temp_decay_pulse)

In [ ]:
qubit_decay_pulses[1].draw(backend=backend)

ここでハードウェアに連続パルスを照射する。

In [ ]:
num_shots = 4*1024  # より正確な結果を得るためには、この値を増やす
                    # ただし、この値を増やすと計算時間が長くなる

job_qubit_decay = backend.run(qubit_decay_pulses, 
                              meas_level=1, 
                              meas_return='avg', 
                              shots=num_shots)

job_monitor(job_qubit_decay)

次に、測定データにアクセスします。緩和時間定数を取得するために指数関数でデータをフィットします。

In [ ]:
qubit_decay_results = job_qubit_decay.result(timeout=120)

qubit_decay_values = []
for i in range(len(delay_times)):
    qubit_decay_values.append(qubit_decay_results.get_memory(i)[qubit])
qubit_decay_values = np.abs(qubit_decay_values)
qubit_decay_values = (qubit_decay_values-min(qubit_decay_values))
qubit_decay_values/=max(qubit_decay_values)

decay_popt, _=fit_exponential(delay_times, qubit_decay_values)
tau=decay_popt[0]
g=1/tau

plt.scatter(delay_times*1e6, qubit_decay_values, color='black') 
plt.plot(delay_times*1e6, exponential(delay_times, *decay_popt), '--', lw=2, color='red', label=r'$\tau$={:.1f} $\mu$s'.format(tau*1e6))
plt.title("$T_1$ Experiment", fontsize=15)
plt.xlabel('Delay before measurement [$\mu$s]', fontsize=15)
plt.ylabel('Signal [a.u.]', fontsize=15)
plt.legend()
plt.show()

print(r'gamma=  {:.2f} kHz'.format(g/1e3))

この章ではJaynes-Cummingsモデルを導入し、量子ビットとキャビティが結合したシステムに関連する値について考えました。量子ビットー共振期間の結合強度$g$、量子ビットの自発的な放出率$\gamma$、共振器の緩和率$\kappa$を取得するためにQiskit Pulseを利用しました。これらのパラメータは<a href="https://qiskit.org/textbook/ch-quantum-hardware/cQED-JC-SW.html">以前の章</a>で測定したような量子ビット周波数と共振周波数に密接に関連しており、量子ビットと共振器の結合系に対する解釈を与えます。

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table